In [13]:
import requests
import json
import time
import os

In [14]:
#URL PREPARATION
KOTA_BANDUNG=208977
KEC_HEGARMANAH=3454203

URL="http://dataservice.accuweather.com/currentconditions/v1/{}".format(KOTA_BANDUNG)

KEYS=[
      #add your accuweather api keys here
      ]


In [15]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  database="datacuaca",
  user="<your database user>",
  password="<your password user>"
)


In [16]:
def getTime():
  t=time.localtime()
  current_time = time.strftime("%d-%m-%y %H.%M.%S", t)
  return current_time

In [17]:
def saveToTxt(data):
    waktu=getTime()[3:8]
    os.makedirs(waktu, exist_ok=True) #buat folder kalau belum ada
    with open(waktu+"/"+getTime()+".txt", 'w') as f: 
        for key, value in data.items(): 
            f.write('{%s:%s},\n' % (key, value))

In [18]:
#untuk membuat tabel setiap bulannya
def createTables(name,waktu):
    query="""
    CREATE TABLE `{}-{}` (
  `LocalObservationDateTime` varchar(50) DEFAULT NULL,
  `EpochTime` int(11) DEFAULT NULL,
  `WeatherText` varchar(50) DEFAULT NULL,
  `WeatherIcon` int(11) DEFAULT NULL,
  `HasPrecipitation` tinyint(1) DEFAULT NULL,
  `PrecipitationType` varchar(25) DEFAULT NULL,
  `IsDayTime` tinyint(1) DEFAULT NULL,
  `TemperatureMetricValue` double DEFAULT NULL COMMENT 'dalam celsius',
  `RealFeelTemperature` double DEFAULT NULL COMMENT 'dalam celsius',
  `RealFeelTemperaturePhrase` varchar(20) DEFAULT NULL,
  `RealFeelTemperatureShade` int(11) DEFAULT NULL COMMENT 'dalam celsius',
  `RealFeelTemperatureShadePharase` varchar(20) DEFAULT NULL,
  `RelativeHumidity` int(11) DEFAULT NULL,
  `IndoorRelativeHumidity` int(11) DEFAULT NULL,
  `DewPoint` double DEFAULT NULL COMMENT 'dalam celsius',
  `WindDirDegrees` int(11) DEFAULT NULL COMMENT 'dalam derajat',
  `WindSpeed` double DEFAULT NULL COMMENT 'dalam km/h',
  `WindGustSpeed` double DEFAULT NULL COMMENT 'dalam km/h',
  `UVIndex` int(11) DEFAULT NULL,
  `UVIndexText` varchar(20) DEFAULT NULL,
  `Visibility` double DEFAULT NULL COMMENT 'dalam km',
  `ObstructionsToVisibility` int(11) DEFAULT NULL,
  `CloudCover` int(11) DEFAULT NULL,
  `Ceiling` int(11) DEFAULT NULL COMMENT 'dalam m',
  `Pressure` double NULL COMMENT 'dalam mb',
  `PressureTendency` varchar(20) NULL,
  `Past24HourTemperatureDeparture` double DEFAULT NULL COMMENT 'dalam celcius',
  `ApparentTemperature` double DEFAULT NULL COMMENT 'dalam celcius',
  `WindChillTemperature` double NULL COMMENT 'dalam celcius',
  `WetBulbTemperature` double NULL COMMENT 'dalam celcius',
  `Precip1hr` double NULL COMMENT 'dalam mm',
  `PrecipitationSummary` double NULL COMMENT 'dalam mm',
  `PrecipitationSummaryPastHour` double NULL COMMENT 'dalam mm',
  `PrecipitationSummaryPast3Hours` double NULL COMMENT 'dalam mm',
  `PrecipitationSummaryPast6Hours` double NULL COMMENT 'dalam mm',
  `PrecipitationSummaryPast9Hours` double NULL COMMENT 'dalam mm',
  `PrecipitationSummaryPast12Hours` double NULL COMMENT 'dalam mm',
  `PrecipitationSummaryPast18Hours` double NULL COMMENT 'dalam mm',
  `PrecipitationSummaryPast24Hours` double NULL COMMENT 'dalam mm',
  `TemperatureSummaryPast6HourRangeMin` double NULL COMMENT 'dalam celcius',
  `TemperatureSummaryPast6HourRangeMax` double NULL COMMENT 'dalam celcius',
  `TemperatureSummaryPast12HourRangeMin` double NULL COMMENT 'dalam celcius',
  `TemperatureSummaryPast12HourRangeMax` double NULL COMMENT 'dalam celcius',
  `TemperatureSummaryPast24HourRangeMin` double NULL COMMENT 'dalam celcius',
  `TemperatureSummaryPast24HourRangeMax` double NULL COMMENT 'dalam celcius'
)""".format(name,waktu)

    mycursor = mydb.cursor()
    mycursor.execute(query) 

In [19]:
def insertQuery(name,waktu):
    query="""
        INSERT INTO `{}-{}` (
        LocalObservationDateTime, 
        EpochTime,
        WeatherText,
        WeatherIcon,
        HasPrecipitation,
        PrecipitationType,
        IsDayTime,
        TemperatureMetricValue,
        RealFeelTemperature,
        RealFeelTemperaturePhrase,
        RealFeelTemperatureShade,
        RealFeelTemperatureShadePharase,
        RelativeHumidity,
        IndoorRelativeHumidity,
        DewPoint,
        WindDirDegrees,
        WindSpeed,
        WindGustSpeed,
        UVIndex,
        UVIndexText,
        Visibility,
        ObstructionsToVisibility,
        CloudCover,
        Ceiling,
        Pressure,
        PressureTendency,
        Past24HourTemperatureDeparture,
        ApparentTemperature,
        WindChillTemperature,
        WetBulbTemperature,
        Precip1hr,
        PrecipitationSummary,
        PrecipitationSummaryPastHour,
        PrecipitationSummaryPast3Hours,
        PrecipitationSummaryPast6Hours,
        PrecipitationSummaryPast9Hours,
        PrecipitationSummaryPast12Hours,
        PrecipitationSummaryPast18Hours,
        PrecipitationSummaryPast24Hours,
        TemperatureSummaryPast6HourRangeMin,
        TemperatureSummaryPast6HourRangeMax,
        TemperatureSummaryPast12HourRangeMin,
        TemperatureSummaryPast12HourRangeMax,
        TemperatureSummaryPast24HourRangeMin,
        TemperatureSummaryPast24HourRangeMax
        ) 
        VALUES (%s, %s, %s, %s, %s, 
                %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s)
            """.format(name,waktu)
    return query

In [20]:
def prepDataDB(data):
    result=(
    data['LocalObservationDateTime'],
        data['EpochTime'],
        data['WeatherText'],
        data['WeatherIcon'],
        data['HasPrecipitation'],
        
        data['PrecipitationType'],
        data['IsDayTime'],
        data['Temperature']['Metric']['Value'],
        data['RealFeelTemperature']['Metric']['Value'],
        
        data['RealFeelTemperature']['Metric']['Phrase'],
        data['RealFeelTemperatureShade']['Metric']['Value'],
        data['RealFeelTemperatureShade']['Metric']['Phrase'],
        data['RelativeHumidity'],
        data['IndoorRelativeHumidity'],
        data['DewPoint']['Metric']['Value'],
                                              
        data['Wind']['Direction']['Degrees'],
        data['Wind']['Speed']['Metric']['Value'],
        data['WindGust']['Speed']['Metric']['Value'],
        data['UVIndex'],
        data['UVIndexText'],
                                              
        data['Visibility']['Metric']['Value'],
        data['ObstructionsToVisibility'],
        data['CloudCover'],
        data['Ceiling']['Metric']['Value'],
        data['Pressure']['Metric']['Value'],
                                              
        data['PressureTendency']['LocalizedText'],
        data['Past24HourTemperatureDeparture']['Metric']['Value'],
        data['ApparentTemperature']['Metric']['Value'],
        data['WindChillTemperature']['Metric']['Value'],
        data['WetBulbTemperature']['Metric']['Value'],
                                              
        data['Precip1hr']['Metric']['Value'],
        data['PrecipitationSummary']['Precipitation']['Metric']['Value'],
        data['PrecipitationSummary']['PastHour']['Metric']['Value'],
        data['PrecipitationSummary']['Past3Hours']['Metric']['Value'],
        data['PrecipitationSummary']['Past6Hours']['Metric']['Value'],
                                              
        data['PrecipitationSummary']['Past9Hours']['Metric']['Value'],
        data['PrecipitationSummary']['Past12Hours']['Metric']['Value'],
        data['PrecipitationSummary']['Past18Hours']['Metric']['Value'],
        data['PrecipitationSummary']['Past24Hours']['Metric']['Value'],
        data['TemperatureSummary']['Past6HourRange']['Minimum']['Metric']['Value'],
                                              
        data['TemperatureSummary']['Past6HourRange']['Maximum']['Metric']['Value'],
        data['TemperatureSummary']['Past12HourRange']['Minimum']['Metric']['Value'],
        data['TemperatureSummary']['Past12HourRange']['Maximum']['Metric']['Value'],
        data['TemperatureSummary']['Past24HourRange']['Minimum']['Metric']['Value'],
        data['TemperatureSummary']['Past24HourRange']['Maximum']['Metric']['Value']
    )

    return result
                                              


In [21]:
def saveToDB(data):
    
    waktu=getTime()[3:8]
    queryCheck="SHOW TABLES LIKE 'bandung-{}'".format(waktu)
    
    #kalau tidak ada
    mycursor = mydb.cursor()
    mycursor.execute(queryCheck)
    result = mycursor.fetchone()
    if not result:
        createTables("bandung",waktu)
    
    
    #lanjut isi data
    query=insertQuery("bandung",waktu)
    valuesDB=prepDataDB(data)
    mycursor.execute(query, valuesDB)

    mydb.commit()
    
#     query="INSERT INTO "


In [22]:
def saveData(data):
    data=json.loads(data)
    data=data[0]
    saveToTxt(data)
    saveToDB(data)
    return data['PrecipitationType']

In [23]:
#loop\

def main(loop):
    index=0;
    while(True):
        hujan="error"
        parameters={"apikey":KEYS[index],"language":"en-us","details":"true"}
        
        try:
            response=requests.get(URL,parameters,timeout=10)
            responseCode=response.status_code
        except:
            time.sleep(5)
            main(loop)
            
        index+=1
        if(responseCode==200):
            error=saveData(response.text)
            if(error==None):
                error="tidak"
                print("{}. Status code: {} rain: {} time: {}".format(loop,response.status_code,error,getTime()))
                loop+=1

        else:
            print("Status code: {} time: {}".format(response.status_code,getTime()))
            continue;




        time.sleep(300)

        if(index==len(KEYS)):
            index=0;

In [24]:
loop=0
main(loop)


0. Status code: 200 rain: tidak time: 10-11-22 09.11.16
1. Status code: 200 rain: tidak time: 10-11-22 09.16.17
2. Status code: 200 rain: tidak time: 10-11-22 09.21.18
3. Status code: 200 rain: tidak time: 10-11-22 09.26.19
4. Status code: 200 rain: tidak time: 10-11-22 09.31.20
5. Status code: 200 rain: tidak time: 10-11-22 09.46.25
6. Status code: 200 rain: tidak time: 10-11-22 09.51.27
7. Status code: 200 rain: tidak time: 10-11-22 09.56.31
8. Status code: 200 rain: tidak time: 10-11-22 10.01.31
9. Status code: 200 rain: tidak time: 10-11-22 10.06.32
10. Status code: 200 rain: tidak time: 10-11-22 10.11.33
11. Status code: 200 rain: tidak time: 10-11-22 10.16.35
12. Status code: 200 rain: tidak time: 10-11-22 10.21.36
13. Status code: 200 rain: tidak time: 10-11-22 10.26.37
14. Status code: 200 rain: tidak time: 10-11-22 10.31.38
15. Status code: 200 rain: tidak time: 10-11-22 10.36.38
16. Status code: 200 rain: tidak time: 10-11-22 10.41.39
17. Status code: 200 rain: tidak time: 10